**3 Stock Porfolio Backtesting**

Backtesting/ Optimization for 3 stock portfoliio:
*   VTI - Vanguard Total Stock Market ETF	Large Cap Growth Equities/ Large Cap Growth
*   VXUS - Vanguard Total International Stock ETF/ Foreign Large Cap
*   BND - Vanguard Total Bond Market ETF/ Total Bonds

In [111]:
# !pip install yfinance --upgrade
# !curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
# !cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
# !pip install PyPortfolioOpt
# !pip install pandas-datareader

In [128]:
import os, sys, copy
import time
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf
import talib as ta

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pandas_datareader import data as pdr

plt.style.use("fivethirtyeight")
pd.options.display.float_format = '{:,}'.format
yf.pdr_override()

**1) Defining Data Folder Structures & Parameters**

In [247]:
# 3 stock + benchmark ticker
ticker_list = ['VTI', 'VXUS', 'BND', '^GSPC']

# define date range
start_date = '2019-01-01'
end_date = '2023-12-31'
interval = '1d'

# define portfolio mix


# initial capital in USD
initial_capital = 100000

# position sizing
pos_size = round(initial_capital/len(ticker_list),2)

**2) Getting OHLC Data from Yahoo Finance** </b>


*   Get stock data based on ticker list from Yahoo Finance
*   Create each stock data as different dataframe
*   Add Technical Analysis indicators from TA-Lib
*   Pointing working directory to data folder, export dach dataframe as independent csv file





In [248]:
# getting OHLC data from yfinance package, if auto_adjust=True, OHLC data will not have adj close column, use progress=False to get rid of comments
# use %whos to list out all the DataFrame in session

for ticker in ticker_list:
  try:
    globals()[ticker] = yf.download(ticker, start=start_date, end=end_date, interval=interval, auto_adjust=True, back_adjust=True, progress=False)
    globals()[ticker] = globals()[ticker].round(4).reset_index()

    # for backtesting
    globals()[ticker][['Position', 'Win Count']] = 0
    globals()[ticker][['Lot Size', 'Equity Value', 'MDD_dollar', 'PNL', 'Holding Period']] = np.NAN

    # strip out special characters in dataframe name, assign new name to df
    if '^' in ticker:
      new_name = ticker.strip('^')
    else:
      new_name = ticker
      globals()[new_name] = globals()[ticker]
  except:
    print('No Data: ', globals()[ticker])

**3) Building the DataFrame List, Perform Backtesting**

In [249]:
# list of dataframes to be iterated after yfinance data pull
df_list = [VTI, VXUS, BND, GSPC]

In [250]:
# function to extract dataframe name
def get_df_name(df):
   name =[x for x in globals() if globals()[x] is df][0]
   return name

In [251]:
# function to print function name as a string
def get_func_name(func_name):
    func_name = sys._getframe().f_code.co_name
    return func_name

**4) PyPortfolioOpt Optinization Results**

In [252]:
SP500_open = GSPC.iloc[0]['Close']
SP500_close = GSPC.iloc[-1]['Close']
benchmark_return = SP500_close/SP500_open -1
print('S&P500 benchmark return for same period: ', benchmark_return.round(3))

S&P500 benchmark return for same period:  0.9


In [253]:
# creating data required for pyportfolio
assets = ['VTI', 'VXUS', 'BND']

#assign weight to each stock
weights = np.array([0.65,0.25,0.1])

# get data via pandas datareader
df = pd.DataFrame()
for stock in assets:
    df[stock] = pdr.get_data_yahoo(stock, start =start_date, end=end_date, progress=False)['Adj Close'].round(4)

# getting the basic math input done
returns     = df.pct_change()
returns_std = returns.std().round(4)
cov_matrix_annual = returns.cov() * 252
port_variance     = np.dot(weights.T, np.dot(cov_matrix_annual, weights))
port_volatility   = np.sqrt(port_variance)
portfolioSimpleAnnualReturn = np.sum(returns.mean()* weights)*252

In [254]:
# compute result based on initial allocation
percent_var = str(round(port_variance,2)*100) + "%"
percent_vols = str(round(port_volatility,2)*100) + "%"
percent_ret = str(round(portfolioSimpleAnnualReturn,2)*100) + "%"

print('Result based on original allocation')
print('Weight - VTI/VXUS/BND: ', weights.round(2))
print('Expected annual return: ', percent_ret)
print('Annual volatility/risk: ', percent_vols)
print('Annual variance: ', percent_var)

Result based on original allocation
Weight - VTI/VXUS/BND:  [0.65 0.25 0.1 ]
Expected annual return:  13.0%
Annual volatility/risk:  19.0%
Annual variance:  3.0%


Weight - VTI/VXUS/BND:  [0.34 0.33 0.33]
Expected annual return:  9.0%
Annual volatility/risk:  14.02%
Annual variance:  2.0%

Weight - VTI/VXUS/BND:  [0.4 0.4 0.2]
Expected annual return:  10.0%
Annual volatility/risk:  16.0%
Annual variance:  3.0%

Weight - VTI/VXUS/BND:  [0.45 0.45 0.1 ]
Expected annual return:  12.0%
Annual volatility/risk:  18.0%
Annual variance:  3.0%

Weight - VTI/VXUS/BND:  [0.65 0.25 0.1 ]
Expected annual return:  13.0%
Annual volatility/risk:  19.0%
Annual variance:  3.0%

Weight - VTI/VXUS/BND:  [0.8  0.15 0.05]
Expected annual return:  15.0%
Annual volatility/risk:  20.0%
Annual variance:  4.0%

S&P500 benchmark annual return for same period 18%

In [255]:
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

#optimize for max sharpe ratio
ef = EfficientFrontier(mu , S)
weights = ef.max_sharpe()

print('Assuming max return:')
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose= True)
print(cleaned_weights)

Assuming max return:
Expected annual return: 15.1%
Annual volatility: 21.6%
Sharpe Ratio: 0.61
OrderedDict([('VTI', 1.0), ('VXUS', 0.0), ('BND', 0.0)])
